In [207]:
import pandas as pd
import re

# regulare expression
float_exp = re.compile(r'[0-9.]+')
int_exp = re.compile(r'[0-9]+')
str_exp = re.compile(r'[a-zA-Z_.+-]')

#Data cleaning
df = pd.read_csv('emp.txt', skipinitialspace = True, dtype=str)
df = (df.replace('\t','', regex=True).replace(r'^\s*$', "0", regex=True))
df.columns = [c.strip().lower().replace(r'^\s*$', '') for c in df.columns]

#removing missing data raws
error_df = df[df.isna().any(axis=1)]
error_df.insert(0, 'error_message', "The number of columns in the record doesn't match file header spec.")
df = df.dropna()

#validating the datatype
for index, row in list(df.iterrows())[3:]:
    key_list =  list()
    if not str_exp.search(row['name']):
        key_list.append('name')
    if not int_exp.search(row['age']):
        key_list.append('age')
    if not float_exp.search(row['salary']):
        key_list.append('salary')
    if not int_exp.search(row['benefits']):
        key_list.append('benefits')
    if not str_exp.search(row['department']):
        key_list.append('department')
    if len(key_list)>0:
        error_dict = {"name":row['name'],"age":row['age'], "salary": row['salary'], "benefits":row['benefits'], "department":row['department'] }
        error_dict["error_message"] = "The datatypes of columns:["+",".join(key_list)+"] doesn't match the datatypes specified in the first test record."
        data = pd.DataFrame(error_dict,  index=[0])
        error_df = error_df.append(data)
        df.drop(index, inplace = True )
        df.drop(0, inplace = True )

convert_dict = {'name': str,
                'age': int,
                'salary': float,
                'benefits': int,
                'department': str
               }

#data type assign on clean data
df = df.astype(convert_dict)
df['salary']=df['salary'].apply(lambda x:round(x,2))

# writing data to files
df.to_csv('output.txt', index=False)
error_df.to_csv('quarantine.txt', index=False)